In [7]:
import llm

In [2]:
source_code = """
 !------------------------------------------------------------------------------
  SUBROUTINE ci_func(ci, fval, p, iv, c, gb_mol, je, cair, oair, lmr_z, par_z, rh_can, gs_mol, atm2lnd_inst, photosyns_inst)
  !
  !! DESCRIPTION:
  ! evaluate the function
  ! f(ci)=ci - (ca - (1.37rb+1.65rs))*patm*an
  !
  ! remark:  I am attempting to maintain the original code structure, also
  ! considering one may be interested to output relevant variables for the
  ! photosynthesis model, I have decided to add these relevant variables to
  ! the relevant data types.
  !
  !!ARGUMENTS:
  REAL(KIND = r8), INTENT(IN) :: ci
  ! intracellular leaf CO2 (Pa)
  REAL(KIND = r8), INTENT(IN) :: lmr_z
  ! canopy layer: leaf maintenance respiration rate (umol CO2/m**2/s)
  REAL(KIND = r8), INTENT(IN) :: par_z
  ! par absorbed per unit lai for canopy layer (w/m**2)
  REAL(KIND = r8), INTENT(IN) :: gb_mol
  ! leaf boundary layer conductance (umol H2O/m**2/s)
  REAL(KIND = r8), INTENT(IN) :: je
  ! electron transport rate (umol electrons/m**2/s)
  REAL(KIND = r8), INTENT(IN) :: cair
  ! Atmospheric CO2 partial pressure (Pa)
  REAL(KIND = r8), INTENT(IN) :: oair
  ! Atmospheric O2 partial pressure (Pa)
  REAL(KIND = r8), INTENT(IN) :: rh_can
  ! canopy air realtive humidity
  INTEGER, INTENT(IN) :: p, iv, c
  ! pft, vegetation type and column indexes
  REAL(KIND = r8), INTENT(OUT) :: fval
  ! return function of the value f(ci)
  REAL(KIND = r8), INTENT(OUT) :: gs_mol
  ! leaf stomatal conductance (umol H2O/m**2/s)
  TYPE(atm2lnd_type), INTENT(IN) :: atm2lnd_inst
  TYPE(photosyns_type), INTENT(INOUT) :: photosyns_inst
  !
  !local variables
  REAL(KIND = r8) :: ai
  ! intermediate co-limited photosynthesis (umol CO2/m**2/s)
  REAL(KIND = r8) :: cs
  ! CO2 partial pressure at leaf surface (Pa)
  REAL(KIND = r8) :: term
  ! intermediate in Medlyn stomatal model
  REAL(KIND = r8) :: aquad, bquad, cquad
  ! terms for quadratic equations
  REAL(KIND = r8) :: r1, r2
  ! roots of quadratic equation
  !------------------------------------------------------------------------------

  ASSOCIATE(forc_pbot => atm2lnd_inst % forc_pbot_downscaled_col, c3flag => photosyns_inst % c3flag_patch, ivt => patch % itype, medlynslope => pftcon % medlynslope, medlynintercept => pftcon % medlynintercept, stomatalcond_mtd => photosyns_inst % stomatalcond_mtd, ac => photosyns_inst % ac_patch, aj => photosyns_inst % aj_patch, ap => photosyns_inst % ap_patch, ag => photosyns_inst % ag_patch, an => photosyns_inst % an_patch, vcmax_z => photosyns_inst % vcmax_z_patch, cp => photosyns_inst % cp_patch, kc => photosyns_inst % kc_patch, ko => photosyns_inst % ko_patch, qe => photosyns_inst % qe_patch, tpu_z => photosyns_inst % tpu_z_patch, kp_z => photosyns_inst % kp_z_patch, bbb => photosyns_inst % bbb_patch, mbb => photosyns_inst % mbb_patch)
    ! Output: [real(r8) (:)   ]  atmospheric pressure (Pa)
      ! Output: [logical  (:)   ]  true if C3 and false if C4
      ! Input:  [integer  (:)   ]  patch vegetation type
      ! Input:  [real(r8) (:)   ]  Slope for Medlyn stomatal conductance model method
      ! Input:  [real(r8) (:)   ]  Intercept for Medlyn stomatal conductance model method
      ! Input:  [integer        ]  method type to use for stomatal conductance
      ! Output: [real(r8) (:,:) ]  Rubisco-limited gross photosynthesis (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  RuBP-limited gross photosynthesis (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  product-limited (C3) or CO2-limited (C4) gross photosynthesis (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  co-limited gross leaf photosynthesis (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  net leaf photosynthesis (umol CO2/m**2/s)
      ! Input:  [real(r8) (:,:) ]  maximum rate of carboxylation (umol co2/m**2/s)
      ! Output: [real(r8) (:)   ]  CO2 compensation point (Pa)
      ! Output: [real(r8) (:)   ]  Michaelis-Menten constant for CO2 (Pa)
      ! Output: [real(r8) (:)   ]  Michaelis-Menten constant for O2 (Pa)
      ! Output: [real(r8) (:)   ]  quantum efficiency, used only for C4 (mol CO2 / mol photons)
      ! Output: [real(r8) (:,:) ]  triose phosphate utilization rate (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  initial slope of CO2 response curve (C4 plants)
      ! Output: [real(r8) (:)   ]  Ball-Berry minimum leaf conductance (umol H2O/m**2/s)
      ! Output: [real(r8) (:)   ]  Ball-Berry slope of conductance-photosynthesis relationship

      IF (c3flag(p)) THEN
      ! C3: Rubisco-limited photosynthesis
      ac(p, iv) = vcmax_z(p, iv) * MAX(ci - cp(p), 0._r8) / (ci + kc(p) * (1._r8 + oair / ko(p)))

      ! C3: RuBP-limited photosynthesis
      aj(p, iv) = je * MAX(ci - cp(p), 0._r8) / (4._r8 * ci + 8._r8 * cp(p))

      ! C3: Product-limited photosynthesis
      ap(p, iv) = 3._r8 * tpu_z(p, iv)

    ELSE

      ! C4: Rubisco-limited photosynthesis
      ac(p, iv) = vcmax_z(p, iv)

      ! C4: RuBP-limited photosynthesis
      aj(p, iv) = qe(p) * par_z * 4.6_r8

      ! C4: PEP carboxylase-limited (CO2-limited)
      ap(p, iv) = kp_z(p, iv) * MAX(ci, 0._r8) / forc_pbot(c)

    END IF

    ! Gross photosynthesis. First co-limit ac and aj. Then co-limit ap

    aquad = params_inst % theta_cj(ivt(p))
    bquad = - (ac(p, iv) + aj(p, iv))
    cquad = ac(p, iv) * aj(p, iv)
    CALL quadratic(aquad, bquad, cquad, r1, r2)
    ai = MIN(r1, r2)

    aquad = params_inst % theta_ip
    bquad = - (ai + ap(p, iv))
    cquad = ai * ap(p, iv)
    CALL quadratic(aquad, bquad, cquad, r1, r2)
    ag(p, iv) = MAX(0._r8, MIN(r1, r2))

    ! Net photosynthesis. Exit iteration if an < 0

    an(p, iv) = ag(p, iv) - lmr_z
    IF (an(p, iv) < 0._r8) THEN
      fval = 0._r8
      RETURN
    END IF
    ! Quadratic gs_mol calculation with an known. Valid for an >= 0.
    ! With an <= 0, then gs_mol = bbb or medlyn intercept
    cs = cair - 1.4_r8 / gb_mol * an(p, iv) * forc_pbot(c)
    cs = MAX(cs, max_cs)
    IF (stomatalcond_mtd == stomatalcond_mtd_medlyn2011) THEN
      term = 1.6_r8 * an(p, iv) / (cs / forc_pbot(c) * 1.E06_r8)
      aquad = 1.0_r8
      bquad = - (2.0 * (medlynintercept(patch % itype(p)) * 1.E-06_r8 + term) + (medlynslope(patch % itype(p)) * term) ** 2 / (gb_mol * 1.E-06_r8 * rh_can))
      cquad = medlynintercept(patch % itype(p)) * medlynintercept(patch % itype(p)) * 1.E-12_r8 + (2.0 * medlynintercept(patch % itype(p)) * 1.E-06_r8 + term * (1.0 - medlynslope(patch % itype(p)) * medlynslope(patch % itype(p)) / rh_can)) * term

      CALL quadratic(aquad, bquad, cquad, r1, r2)
      gs_mol = MAX(r1, r2) * 1.E06_r8
    ELSE IF (stomatalcond_mtd == stomatalcond_mtd_bb1987) THEN
      aquad = cs
      bquad = cs * (gb_mol - bbb(p)) - mbb(p) * an(p, iv) * forc_pbot(c)
      cquad = - gb_mol * (cs * bbb(p) + mbb(p) * an(p, iv) * forc_pbot(c) * rh_can)
      CALL quadratic(aquad, bquad, cquad, r1, r2)
      gs_mol = MAX(r1, r2)
    END IF




    ! Derive new estimate for ci

    fval = ci - cair + an(p, iv) * forc_pbot(c) * (1.4_r8 * gs_mol + 1.6_r8 * gb_mol) / (gb_mol * gs_mol)

  END ASSOCIATE

END SUBROUTINE ci_func
"""

In [4]:
fortran_unit_tests = llm._generate_fortran_unit_tests(source_code)

Generating unit tests in Fortran...
PROMPT: 
    Given fortran code, write unit tests using funit.

    Example:
    FORTRAN CODE:
    ```
    module fac
        implicit none
        
        contains

        recursive function factorial(n) result(fact)
            integer, intent(in) :: n
            integer :: fact

            if (n == 0) then
            fact = 1
            else
            fact = n * factorial(n - 1)
            end if
        end function factorial
    end module fac
    ```

    FORTRAN TESTS:
    ```
    @test
    subroutine test_fac()
        use funit

        @assertEqual(120, factorial(5), 'factorial(5)')
        @assertEqual(1, factorial(1), 'factorial(1)')
        @assertEqual(1, factorial(0), 'factorial(0)')

    end subroutine test_fac
    ```

    Your turn:
    FORTRAN CODE:
```

 !------------------------------------------------------------------------------
  SUBROUTINE ci_func(ci, fval, p, iv, c, gb_mol, je, cair, oair, lmr_z, par_z, rh_can, gs_

In [5]:
python_unit_tests = llm._translate_tests_to_python(fortran_unit_tests)

Translating unit tests to Python...
PROMPT: 
    Convert the following unit tests from Fortran to Python using pytest. No need to import the module under test. ```

!------------------------------------------------------------------------------
SUBROUTINE test_ci_func()
  USE funit

  ! Define input variables
  REAL(KIND = r8), PARAMETER :: ci = 400.0_r8
  REAL(KIND = r8), PARAMETER :: lmr_z = 1.0_r8
  REAL(KIND = r8), PARAMETER :: par_z = 100.0_r8
  REAL(KIND = r8), PARAMETER :: gb_mol = 0.1_r8
  REAL(KIND = r8), PARAMETER :: je = 10.0_r8
  REAL(KIND = r8), PARAMETER :: cair = 400.0_r8
  REAL(KIND = r8), PARAMETER :: oair = 210.0_r8
  REAL(KIND = r8), PARAMETER :: rh_can = 0.5_r8
  INTEGER, PARAMETER :: p = 1
  INTEGER, PARAMETER :: iv = 1
  INTEGER, PARAMETER :: c = 1

  ! Define expected output values
  REAL(KIND = r8), PARAMETER :: expected_fval = 0.0_r8
  REAL(KIND = r8), PARAMETER :: expected_gs_mol = 0.0_r8

  ! Define variables for actual output values
  REAL(KIND = r8) :: actu

In [3]:
# Generate a translated Python function
python_function = llm._translate_function_to_python(source_code)

Translating function to Python...
PROMPT: 
    Convert the following Fortran function to Python. ```

 !------------------------------------------------------------------------------
  SUBROUTINE ci_func(ci, fval, p, iv, c, gb_mol, je, cair, oair, lmr_z, par_z, rh_can, gs_mol, atm2lnd_inst, photosyns_inst)
  !
  !! DESCRIPTION:
  ! evaluate the function
  ! f(ci)=ci - (ca - (1.37rb+1.65rs))*patm*an
  !
  ! remark:  I am attempting to maintain the original code structure, also
  ! considering one may be interested to output relevant variables for the
  ! photosynthesis model, I have decided to add these relevant variables to
  ! the relevant data types.
  !
  !!ARGUMENTS:
  REAL(KIND = r8), INTENT(IN) :: ci
  ! intracellular leaf CO2 (Pa)
  REAL(KIND = r8), INTENT(IN) :: lmr_z
  ! canopy layer: leaf maintenance respiration rate (umol CO2/m**2/s)
  REAL(KIND = r8), INTENT(IN) :: par_z
  ! par absorbed per unit lai for canopy layer (w/m**2)
  REAL(KIND = r8), INTENT(IN) :: gb_mol
  ! leaf